In [1]:

import tweepy

client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGPIWwEAAAAANh02yZK%2Bg2Ga9OaIGmo%2FdcBKwI4%3DoBVTm4dbV9EsX06kTvtAz5XjSCK222TAxusnGUposUxAGoEFqg')


In [ ]:
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAGPIWwEAAAAANh02yZK%2Bg2Ga9OaIGmo%2FdcBKwI4%3DoBVTm4dbV9EsX06kTvtAz5XjSCK222TAxusnGUposUxAGoEFqg')

query = "republicans -is:retweet lang:en"
# response = client.search_recent_tweets(query=query, tweet_fields=['author_id'], user_fields=['created_at', 'name'],
#                               max_results=10)

response = client.get_list_members()
for tweet in response.data: 
    